In [ ]:
import Pkg;
Pkg.activate("../")
Pkg.instantiate()
Pkg.add(url = "https://github.com/RalphAS/Pseudospectra.jl")

In [ ]:
using Pseudospectra, Plots

In [ ]:
Pkg.status()

In [ ]:
Pkg.update()

# Our example

We start by defining the Blashke product 
$$
B_{\mu}(z) = \frac{z (\mu - z)} {1 - \bar{\mu} z},
$$
with $\mu = \frac{17\sqrt{2}}{32} e^{i \pi/8}$.

In [ ]:
r = sqrt(2)*17/32 
ϕ = π / 8

max_r = 10.0
μ = r * exp(im * ϕ)
B(z; μ=μ) = (z * (μ - z)) / (1 - μ' * z)

# Non rigorous exploration of the Pseudospectrum

We fix the truncation size for the Galerkin approximation.

In [ ]:
K = 128
N = 2*K+1

We start by writing the Blashke product as an interval map

In [ ]:
S(x) = 0.5 + atan((sin(2 * pi * x) - r * sin(ϕ)) / (cos(2 * pi * x) - r * cos(ϕ))) / pi

In [ ]:
using Plots
plot(S, 0, 1)

In [ ]:
using RigorousInvariantMeasures
FourierBasis = RigorousInvariantMeasures.FourierAdjoint(K, 65536)
P = DiscretizedOperator(FourierBasis, S)

In [ ]:
import IntervalArithmetic
midI = IntervalArithmetic.mid
radI = IntervalArithmetic.radius

midP = midI.(real.(P.L)) + im * midI.(imag.(P.L))
spectralportrait(midP)

# Numerical oracle for the constants

In this section we do some numerical computations to narrow down the set of parameters. Later on, we will use self-validated numerical methods (Interval Arithmetic) to certify the 
numerical values we computed now. We do this since numerical computations are inexpensive, while self validated methods may be more time-consuming.

In [ ]:
using Plots

In [ ]:
Pkg.status()

Let $A_{r} = \{z \mid e^{-2\pi r}\leq |z| \leq e^{2\pi r}\}$.

We are interested in finding $\eta$, $\rho$ such that the closure $A_{\rho}$ is contained in $B_{\mu}(A_{\eta})$.
We are interested in maximizing $\alpha-\eta$, since it is the constant appearing in the main error term of our functional analytic treatment, i.e.:
$$
||Lf-L_Kf||_{\mathcal{A}_0}\leq \left(1+\frac{2}{e^{2 \pi (\rho-\alpha)}-1}\right)\left(e^{-2\pi K\alpha}+e^{-2\pi K(\alpha-\eta)}\right)||f||_{\mathcal{A}_{\alpha}}.	
$$

For $\eta>1$ fix
$$
\rho_o(\eta):=\frac{1}{2\pi}\log\left(\min_{\theta \in [0,1]}|B_{\mu}(e^{2\pi\eta} e^{2\pi i \theta})|\right)
$$
where $_0$ stays for outer.
We would like to maximize this function.

In [ ]:
ρ_o(η) = log(minimum(abs.(B.([exp(2 * π*(η + im * θ)) for θ in 0:0.001:1]))))/(2*π)

In [ ]:
plot(η->ρ_o(η)-η, 1, 10)

Similarly, we would like to treat the image inside the circle; for $\eta>1$ we define 
$$
\rho_i(\eta) :=-\frac{1}{2\pi}\log\left(\max_{\theta \in [0, 1]}\left|B_{\mu}(e^{2\pi (-\eta + i \theta)})\right|\right)
$$

In [ ]:
ρ_i(η) = -log(maximum(abs.(B.([exp(2 * π*(-η + im * θ)) for θ in 0:0.001:1]))))/(2*π)

In [ ]:
plot(η->ρ_i(η)-η, 0, 10)

We define now 
$$
\rho(\eta) = \min\{\rho_i(\eta),\rho_o(\eta)\}-\eta
$$
we have that our dynamic is expanding the annulus when this function is positive.

In [ ]:
ρ(η) = min(ρ_i(η), ρ_o(η))

We plot now the function 
$$
    \eta \mapsto \rho(\eta)-\eta
$$
our dynamic is expanding the annulus when this function is positive.

In [ ]:
plot(η -> (ρ(η)-η), 0, 5)

From this non certified plot, we can see that the the difference $\rho(\eta)-\eta$ seems to converge to an asymptotic value bigger than 
$0.04551$.

We need now to be careful in choosing $\eta$ and $\rho$ in such a way that our computation is well behaved.

By our numerical exploration of the pseudospectrum, we want to isolate the eigenvalues outside of a circle of radius $0.5$.

By Lemma 3.10 and Lemma 3.13, we have that 
$$
\frac{||f||_{\mathcal{A}_{\alpha}}}{||f||_{\mathcal{A}_{0}}}\leq 2^{\frac{\alpha}{\alpha-\eta}} \left(1+\frac{2}{e^{2 \pi (\rho - \alpha)} - 1}\right)^{\frac{\alpha}{\alpha-\eta}} 
$$
and
$$
||\mathcal{L}-\mathcal{L}_K||_{\mathcal{A}_{\alpha}\to \mathcal{A}_0}\leq \left(1+\frac{2}{e^{2 \pi (\rho - \alpha)} - 1}\right)\left(e^{-2\pi K\alpha}+e^{-2\pi K(\alpha-\eta)}\right).
$$

We refer to Proposition 2 in the paper, what we would like to control and make small is
$$
||\mathcal{L}-\mathcal{L}_K||_{\mathcal{A}_{\alpha}\to \mathcal{A}_0}\frac{||f||_{\mathcal{A}_{\alpha}}}{||f||_{\mathcal{A}_{0}}};
$$
since $\rho>\alpha$ we have then that $\alpha-\eta$ is at most $0.04551$.

To optimize this, we pass to the logarithm.

In [ ]:
bound(η, ρ, α; K, μ) =  α/(α-η)*log(1/μ)+(α/(α-η)+1)*log(1+2/(exp(2*π*(ρ-α))-1))+log(exp(-2*π*K*α)+exp(-2*π*K*(α-η)))

In [ ]:
bound_η_α(η, α; K, μ) = bound(η, ρ(η), α; K, μ)

In [ ]:
function bound_η(η; K, μ)
    η_eps = η+(ρ(η)-η)/100
    ρ_eps = ρ(η)-(ρ(η)-η)/100
    return minimum([bound_η_α(η, α; K, μ) for α in LinRange(η_eps, ρ_eps, 100)])
end

In [ ]:
bound_η_log2(η; K, μ) = bound_η(η; K, μ)/log(2)

In [ ]:
plot(η -> bound_η_log2(η; K = 256, μ = 1/2), 0.2, 0.5, label = "256")
plot!(η -> bound_η_log2(η; K = 128, μ = 1/2), 0.2, 0.5, label = "128")
plot!(η -> bound_η_log2(η; K = 64, μ = 1/2), 0.2, 0.5, label = "64")
plot!(η -> 53, label = "53 = Float64 Mantissa")

In [ ]:
val, index = findmin([bound_η_log2(η; K = 64, μ = 1/2) for η in LinRange(0.2, 0.5, 10000)])

We choose a discretization size of $K=256$. For this discretization size we fix $\eta = $

# Certifying the constants

For the specific values of $\alpha$, $\rho$ and $\eta$ computed above, we will certify the value of the constants.

In [ ]:
LinRange(0.2, 0.5, 10000)[index]